In [ ]:
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import pymysql

# database 연결 객체
DB_CONN = None

# request한 후 BeautifulSoup return
def get_html(url, params=None):
    response = requests.get(url=url, params=params)
    html = None

    if response.status_code == 200 and (params != None or (params == None and url == response.url)):
        html = BeautifulSoup(response.text, "lxml")
        print("* request성공 >> ", response.url)
    else:
        print("* request실패(혹은 안함)! >> ", str(response.status_code))
        print("* request실패(혹은 안함)! >> ", response.url)

    return html

# xml에서 데이터 추출하여 dict형으로 return
def get_result_to_dict(result_xml, xml_cols):
    result_data = {}
    
    for tag, table_col in xml_cols.items():
        try:
            xml_data = result_xml.find("./body/items/item/" + tag).text
            
            if xml_data != "" and xml_data != "\n":
                result_data[table_col] = xml_data
        except:
            continue
            
    return result_data

# database 연결
def connect_db():
    global DB_CONN
    DB_CONN = pymysql.connect(host='127.0.0.1',
                              user='root',
                              port=3306,
                              password='1q2w3e4r5t!', # database 접속 비밀번호
                              db='jtodb',
                              charset='utf8mb4',
                              use_unicode=True,
                              cursorclass=pymysql.cursors.DictCursor)
    
# 테이블에 데이터 입력
def insert_tour_place_data(xml_data):
    cur = DB_CONN.cursor()
    sql = "INSERT INTO tour_place({}) VALUES ({})".format(",".join(xml_data.keys()), ", ".join(["%s"] * len(xml_data)))
    
    cur.execute(sql, tuple(xml_data.values()))
    
    print("  sql >> ", sql)
    print("  data >> ", tuple(xml_data.values()))
    
    DB_CONN.commit()
    

API_BASE_URL = "http://api.visitkorea.or.kr"
url_params = {"langtype" : "KOR",
              "arrange" : "A",
              "pageNo" : "1",
              "mode" : "listOk",
              #"contenttypeid" : "12",
              "areacode" : "39"
             }

# 크롤링 시작할 페이지번호
pageNo = 1

# 확인용(데이터베이스 입력건수)
db_count = 0

# 데이터베이스 연결
connect_db()

try:
    while True:
        url_params["pageNo"] = pageNo
        list_html = get_html(API_BASE_URL + "/guide/inforArea.do", params=url_params) # 검색결과 목록 페이지

        item_list = list_html.select("ul.galleryList li a") # 검색결과 목록
        
        # 검색결과가 없는 경우 loop종료(프로그램 종료)
        if len(item_list) <= 0:
            break

        print("# 크롤링 진행할 페이지 >> ", pageNo, end="\n")
        for item in list_html.select("ul.galleryList li a"):
            detail_url = item.get("href") # 관광지 상세보기 페이지 url
            
            # 관광지(12), 문화시설(14), 레포츠(28) 데이터만 크롤링
            if ("typeid=12" in detail_url) or ("typeid=14" in detail_url) or ("typeid=28" in detail_url):
                detail_html = get_html(API_BASE_URL + detail_url)

                result_xml1 = ET.fromstring(detail_html.select("#common .debugCon textarea")[0].text) # 공통정보(장소 공통정보)
                result_xml2 = ET.fromstring(detail_html.select("#intro .debugCon textarea")[0].text)  # 소개정보(각 장소타입마다 다른 정보)
                
                # 공통정보 데이터 관련 컬럼
                xml_mapping_cols1 = {
                    "contenttypeid" : "content_type_id", # 장소유형ID(12=관광지, 14=문화시설, 28=레포츠)
                    "contentid" : "content_id", # 장소ID
                    "title" : "title", # 장소명
                    "cat1" : "category1_code", # 대분류
                    "cat2" : "category2_code", # 중분류
                    "cat3" : "category3_code", # 소분류
                    "addr1" : "full_address", # 전체 주소
                    "addr2" : "detail_address",
                    "zipcode" : "zipcode",
                    "tel" : "tel",
                    "mapx" : "map_x", # 경도
                    "mapy" : "map_y", # 위도
                    "overview" : "overview", # 개요
                    "booktour" : "booktour_yn", # 교과서속여행지여부(1=여행지, 0=해당없음)
                }
                
                # 소개정보 데이터 관련 컬럼
                xml_mapping_cols2 = {
                    "heritage1" : "world_heritage", # 세계문화유산유무
                    "heritage2" : "world_natural_heritage", # 세계자연유산유무
                    "heritage3" : "world_memory_heritage", # 세계기록유산유무
                    "useseason" : "use_season", # 이용시기
                    "spendtime" :  "spend_time", # 관람소요시간
                    "discountinfo" : "discount_info", # 할인정보
                    "expguide" : "exp_guide", # 체험안내

                    # 쉬는날
                    "restdate" : "rest_date", # (관광지)
                    "restdateculture" : "rest_date", # (문화시설)
                    "restdateleports" : "rest_date", # (레포츠)

                    # 애완동물 동반 가능여부
                    "chkpet" : "pet", # (관광지)
                    "chkpetculture" : "pet", # (문화시설)
                    "chkpetleports" : "pet", # (레포츠)

                    # 유모차 대여 가능여부
                    "chkbabycarriage" : "baby_carriage", # (관광지)
                    "chkbabycarriageculture" : "baby_carriage", # (문화시설)
                    "chkbabycarriageleports" : "baby_carriage", # (레포츠)

                    # 주차시설
                    "parking" : "parking", # (관광지)
                    "parkingculture" : "parking", # (문화시설)
                    "parkingleports" : "parking", # (레포츠)

                    # 주차요금
                    "parkingfee" : "parking_fee", # (문화시설)
                    "parkingfeeleports" : "parking_fee", # (레포츠)

                    # 이용요금
                    "usefee" : "use_fee", # (문화시설)
                    "usefeeleports" : "use_fee", # (레포츠)

                    # 이용시간
                    "usetime" : "use_time", # (관광지)
                    "usetimeculture" : "use_time", # (문화시설)
                    "usetimeleports" : "use_time", # (레포츠)

                    # 문의 및 안내
                    "infocenter" : "info_center", # (관광지)
                    "infocenterculture" : "info_center", # (문화시설)
                    "infocenterleports" : "info_center", # (레포츠)

                }
                
                # db에 입력할 데이터 dict형으로 변환
                result = get_result_to_dict(result_xml1, xml_mapping_cols1)       # 공통정보 데이터
                result.update(get_result_to_dict(result_xml2, xml_mapping_cols2)) # 소개정보 데이터
                result["org_url"] = API_BASE_URL + detail_url # 키 추가(상세보기 URL:데이터 가져온 웹페이지)
                
                # 데이터베이스에 데이터 입력
                insert_tour_place_data(result)
                db_count += 1
                print("* 현재까지 입력건수 >> ", db_count)
        
        # 크롤링할 다음 페이지값 증가
        pageNo += 1
        
finally:
    # 데이터베이스 연결 해제
    DB_CONN.close()
    print("\n\n##### 프로그램 끝 #####")
                